In [2]:
import pandas as pd
import numpy as np

In [280]:
default=pd.read_csv("../datasets/Default.csv")

In [282]:
default.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [281]:
default['default']=default['default'].apply(lambda x: 0 if x=='No' else 1)
default['student']=default['student'].apply(lambda x: 1 if x=='Yes' else 0)

# Logistic Regression

Note that linear regression won't work on categorical response. Main reasons are (1) the difference between one category to another may not be fixed (2) linear regression outputs value outside of $[0,1]$, making it hard to interpret as probabilities.

Thus, we use a logistic function (aka sigmoid function) to squeeze output values into a valid probability 
distribution

$\sigma(z_i)=\frac{1}{1+e^{-z_i}}$ where $z_i=\beta_{0}+\beta_{1}x_i$

Then $\hat{p}_i=P(y=1|x_i)$

We use Maximum Likelihood Estimate (MLE) to estimate the parameters

$L(\beta_0,\beta_1)=\prod_{i=1}^{n}\hat{p}_i^{y_i}(1-\hat{p}_i)^{1-y_i}$

Then the log-likelihood function becomes

$l(\beta_0,\beta_1)=\sum_{i=1}^{n}y_ilog(\hat{p}_i)+(1-y_i)log(1-\hat{p}_i)$

With some algebra and partial derivatives, we arrive at

$\frac{dl(\beta_0,\beta_1)}{d\beta_0}=\sum_{i=1}^{n}(y_i-\hat{p}_i)$ and
$\frac{dl(\beta_0,\beta_1)}{d\beta_1}=\sum_{i=1}^{n}(y_i-\hat{p}_i)x_i$

Now, update the gradient as follows:

$\beta_0=\beta_0+\eta\sum_{i=1}^{n}(y_i-\hat{p}_i)$

$\beta_1=\beta_1+\eta\sum_{i=1}^{n}(y_i-\hat{p}_i)x_i$, where $\eta$ is the learning rate.

In [283]:
X=default['balance'].values.reshape(-1)

In [284]:
X = (X - np.mean(X)) / np.std(X)

In [285]:
y=default['default'].values

In [286]:
X.shape, y.shape

((10000,), (10000,))

In [287]:
beta_0 = np.random.normal(0, 0.01)
beta_1 = np.random.normal(0, 0.01)
lr=0.0001
n_iter=10000
tol=1e-6
prev_loss=float('inf')

In [288]:
for _ in range(n_iter):
    p_hat=1/(1+np.exp(-(beta_0+beta_1*X)))
    
    loss = -np.mean(y * np.log(p_hat + 1e-15) + (1 - y) * np.log(1 - p_hat + 1e-15))
    # Check convergence
    if abs(prev_loss - loss) < tol:
        print(f"Converged at iteration {i}, loss = {loss:.6f}")
        break
    
    beta_0+=lr*np.sum(y-p_hat)
    beta_1+=lr*np.sum((y-p_hat) * X)

In [289]:
print("b_0 from scratch:",beta_0)
print("b_1 from scratch:",beta_1)

b_0 from scratch: -6.05767351529054
b_1 from scratch: 2.6597755249632526


In [290]:
from sklearn.linear_model import LogisticRegression

In [291]:
lr = LogisticRegression(penalty=None)

In [292]:
lr.fit(X.reshape(-1,1),y)

LogisticRegression(penalty=None)

In [293]:
print("b_0 from sklearn:",lr.intercept_)
print("b_1 from sklearn:",lr.coef_)

b_0 from sklearn: [-6.05769037]
b_1 from sklearn: [[2.65978348]]


# Logistic Regression (Multiple Variables)

In [384]:
default.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [456]:
X=default.drop('default',axis=1).values
y=default['default'].values

In [457]:
X.shape ,y.shape

((10000, 3), (10000,))

In [458]:
X = (X - X.mean(axis=0)) / X.std(axis=0)
X = np.hstack([np.ones((X.shape[0], 1)), X])

In [453]:
beta=np.zeros(X.shape[1])  # one beta per feature, including intercept
lr=0.0001
n_iter=10000
tol=1e-6
prev_loss=float('inf')

In [461]:
for i in range(n_iter):
    z=X@beta
    p_hat=1/(1+np.exp(-z))
    loss = -np.mean(y * np.log(p_hat + 1e-15) + (1 - y) * np.log(1 - p_hat + 1e-15))
    
    # Check convergence
    if abs(prev_loss - loss) < tol:
        print(f"Converged at iteration {i}, loss = {loss:.6f}")
        break
    
    grad = X.T@(y-p_hat)  # shape: (n_features + 1,)
    beta += lr * grad

In [463]:
print("beta from scratch:",beta)

beta from scratch: [-6.16565149 -0.29478268  2.77469481  0.04045401]


In [466]:
lr = LogisticRegression(penalty=None,fit_intercept=False)

In [467]:
lr.fit(X,y)

LogisticRegression(fit_intercept=False, penalty=None)

In [469]:
print("beta from sklearn:",lr.coef_.ravel())

beta from sklearn: [-6.1656557  -0.29478494  2.774699    0.04045078]


# Multinomial Logistic Regression

Recall when $y\in\{0,1\}$, we have $P(y=1|X)=\frac{1}{1+exp(-z)}$ where $z=X^T\beta$

Now suppose $y\in\{1,2,3,...,K\}$ where $K>2$.

Wen need to model a full probability distribution over K classes; that is, $P(y=k|X) \text{ for each } k=1,...,K$.

Softmax function comes into play 

$P(y=k|X)=\frac{exp(X^T\beta_k)}{\sum_{j=1}^{k}exp(X^T\beta_j)}$ and so for each $k$, we have $z_k=X^T\beta_k$.

Identifiability Problem:

We instead model K-1 models to avoid redundancy. We are overestimating parameters because probabilities need to sum to 1.

Without loss of generality,  use Kth class as the baseline model.

Then $P(y=K|X)=\frac{1}{1+\sum_{j=1}^{K-1}exp(X^T\beta_j)}$ and
$P(y=k|X)=\frac{exp(X^T\beta_k)}{1+\sum_{j=1}^{K-1}exp(X^T\beta_j)}$ for each $k=1,...,K-1$

Estimation Set-up:

Given $X\in\mathbb{R}^{N\times(P+1)}$

$y\in\mathbb{R}^{N}$

$B\in\mathbb{R}^{({P+1})\times({K-1})}$



In [283]:
carseats=pd.read_csv("../datasets/Carseats.csv")

In [284]:
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [285]:
K=3

In [286]:
X=carseats.drop(['ShelveLoc','Urban','US'],axis=1).values
y=carseats['ShelveLoc'].astype('category').cat.codes.values #0:Bad 1:Good 3:Medium

In [287]:
X = (X - X.mean(axis=0)) / X.std(axis=0)
X = np.hstack([np.ones((X.shape[0], 1)), X])
B=np.zeros((X.shape[1],K-1))  # one beta per feature, including intercept

In [288]:
def softmax(logits):
    logits_stable = logits - np.max(logits, axis=1, keepdims=True)  # stability trick
    exp_logits = np.exp(logits_stable)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

In [289]:
def compute_loss_and_gradient(X, y, B, K):
    N = X.shape[0]
    
    # Forward pass
    Z = X@B                   # shape: (N, K-1)
    Z_full = np.hstack([Z, np.zeros((N, 1))])  # shape: (N, K)
    probs = softmax(Z_full)      # shape: (N, K)

    # Compute loss
    correct_log_probs = -np.log(probs[np.arange(N), y] + 1e-9)
    loss = np.mean(correct_log_probs)

    # One-hot true labels
    Y = one_hot(y, K)

    # Gradient: only for first K-1 columns
    grad = -(X.T @ (Y[:, :K-1] - probs[:, :K-1])) / N  # shape: (P+1, K-1)

    return loss, grad

In [290]:
def train(X, y, K, lr=0.1, epochs=10000):
    N, P_plus1 = X.shape  # assume X already includes bias column
    B = np.zeros((P_plus1, K - 1))  # initialize weights

    for epoch in range(epochs):
        loss, grad = compute_loss_and_gradient(X, y, B, K)
        B -= lr * grad  # gradient descent update

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {loss:.4f}")

    return B

In [291]:
B=train(X,y,K)

Epoch 0: Loss = 1.0986
Epoch 100: Loss = 0.7714
Epoch 200: Loss = 0.6714
Epoch 300: Loss = 0.6142
Epoch 400: Loss = 0.5756
Epoch 500: Loss = 0.5474
Epoch 600: Loss = 0.5258
Epoch 700: Loss = 0.5088
Epoch 800: Loss = 0.4949
Epoch 900: Loss = 0.4834
Epoch 1000: Loss = 0.4738
Epoch 1100: Loss = 0.4655
Epoch 1200: Loss = 0.4585
Epoch 1300: Loss = 0.4523
Epoch 1400: Loss = 0.4470
Epoch 1500: Loss = 0.4422
Epoch 1600: Loss = 0.4380
Epoch 1700: Loss = 0.4343
Epoch 1800: Loss = 0.4309
Epoch 1900: Loss = 0.4279
Epoch 2000: Loss = 0.4251
Epoch 2100: Loss = 0.4226
Epoch 2200: Loss = 0.4204
Epoch 2300: Loss = 0.4183
Epoch 2400: Loss = 0.4164
Epoch 2500: Loss = 0.4147
Epoch 2600: Loss = 0.4131
Epoch 2700: Loss = 0.4116
Epoch 2800: Loss = 0.4102
Epoch 2900: Loss = 0.4089
Epoch 3000: Loss = 0.4078
Epoch 3100: Loss = 0.4067
Epoch 3200: Loss = 0.4056
Epoch 3300: Loss = 0.4047
Epoch 3400: Loss = 0.4038
Epoch 3500: Loss = 0.4030
Epoch 3600: Loss = 0.4022
Epoch 3700: Loss = 0.4014
Epoch 3800: Loss = 0.400

In [292]:
def predict(X, B):
    Z = X@B
    Z_full = np.hstack([Z, np.zeros((Z.shape[0], 1))])
    probs = softmax(Z_full)
    return np.argmax(probs, axis=1)

In [300]:
B

array([[-2.89416609, -3.85813226],
       [-5.18990202,  6.71835592],
       [ 2.54396455, -3.47180519],
       [ 1.11203446, -0.77230472],
       [ 1.34435925, -1.664928  ],
       [ 0.17309442, -0.03255806],
       [-4.17369657,  5.50344955],
       [-1.546851  ,  1.71269739],
       [-0.08941834,  0.05661568]])

In [303]:
print("Accuracy of MLR from scratch:",accuracy_score(y,predict(X,B)))

Accuracy of MLR from scratch: 0.84


In [294]:
from sklearn.metrics import accuracy_score

In [295]:
from sklearn.linear_model import LogisticRegression

In [296]:
mlr= LogisticRegression(multi_class='multinomial')

In [297]:
mlr.fit(X,y)

LogisticRegression(multi_class='multinomial')

In [305]:
print("Accuracy of MLR with sklearn:",accuracy_score(y,mlr.predict(X)))

Accuracy of MLR with sklearn: 0.8375
